In [3]:
#loading up every rating in data set to Pandas DataFrame
import pandas as pd
from sklearn import neighbors

r_cols=['user_id','movie_id','rating']
ratings=pd.read_csv("C:\\Users\\Ghost\\Desktop\\LearningML\\materials\\ml-100k\\u_data.data",sep='\t',names=r_cols,usecols=range(3),encoding="latin-1")

print(ratings.head())

   user_id  movie_id  rating
0        0        50       5
1        0       172       5
2        0       133       1
3      196       242       3
4      186       302       3


In [4]:
#now we will group everything by MovieID and compute the total number of ratings(each movie's popularity) and the avg of every movie
import numpy as np
movieProperties=ratings.groupby('movie_id').agg({'rating':[np.size,np.mean]})
print(movieProperties.head())

         rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326


In [5]:
#normalize the number of ratings for a movie 0 being noone rating it and 1 being mostly rated
movieNumRatings=pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings=movieNumRatings.apply(lambda x:((x-np.min(x))/(np.max(x)-np.min(x))))
print(movieNormalizedNumRatings.head())

              size
movie_id          
1         0.773585
2         0.222985
3         0.152659
4         0.356775
5         0.145798


In [6]:
#now lets gather genre information
movieDict={}

with open(r"C:\\Users\\Ghost\\Desktop\\LearningML\\materials\\ml-100k\\u_item.item") as f:
    temp =''
    for line in f:
        fields=line.rstrip('\n').split('|')
        movieID=int(fields[0])
        name=fields[1]
        genres=fields[5:25]
        genres = [eval(i) for i in genres]
        #genres=map(int,genres)
        
        movieDict[movieID]=(name,genres,movieNormalizedNumRatings.loc[movieID].get('size'),movieProperties.loc[movieID].rating)
        

In [7]:
movieDict
print(movieDict[2])
print(movieDict[1])

('GoldenEye (1995)', [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], 0.22298456260720412, size    131.000000
mean      3.206107
Name: 2, dtype: float64)
('Toy Story (1995)', [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.7735849056603774, size    452.000000
mean      3.878319
Name: 1, dtype: float64)


In [8]:
#lets define a function that computes distance between two movies based on how similar genres are
#greater the distance the less similar they are
from scipy import spatial

def ComputeDistance(a,b):
    genresA=a[1]
    genresB=b[1]
    genreDistance=spatial.distance.cosine(genresA,genresB)
    popularityA=a[2]
    popularityB=b[2]
    popularityDistance=abs(popularityA-popularityB)
    return genreDistance+popularityDistance
print(ComputeDistance(movieDict[2],movieDict[4]))


0.8004574042309892


In [9]:
import operator

def getNeighbors(movieID,K):
    distances=[]
    for movie in movieDict:
        if(movie!=movieID):
            dist=ComputeDistance(movieDict[movieID],movieDict[movie])
            distances.append((movie,dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors=[]
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors
K=10
avgRating=0
print(movieDict[10])
neighbors=getNeighbors(10,K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0]+" "+str(movieDict[neighbor][3])) 
avgRating /= float(K)

('Richard III (1995)', [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 0.1509433962264151, size    89.000000
mean     3.831461
Name: 10, dtype: float64)
Michael Collins (1996) size    92.000000
mean     3.456522
Name: 744, dtype: float64
Ran (1985) size    70.0
mean     4.1
Name: 647, dtype: float64
Deer Hunter, The (1978) size    120.000000
mean      3.833333
Name: 521, dtype: float64
Killing Fields, The (1984) size    121.000000
mean      4.132231
Name: 528, dtype: float64
Henry V (1989) size    124.000000
mean      4.137097
Name: 190, dtype: float64
Platoon (1986) size    127.000000
mean      3.692913
Name: 157, dtype: float64
Patton (1970) size    136.000000
mean      3.992647
Name: 205, dtype: float64
Paths of Glory (1957) size    33.000000
mean     4.212121
Name: 641, dtype: float64
Seven Years in Tibet (1997) size    155.000000
mean      3.458065
Name: 690, dtype: float64
Welcome To Sarajevo (1997) size    22.000000
mean     3.363636
Name: 1176, dtype: float64


In [10]:
print(avgRating)
print(movieDict[10])

size    100.000000
mean      3.837857
Name: 744, dtype: float64
('Richard III (1995)', [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 0.1509433962264151, size    89.000000
mean     3.831461
Name: 10, dtype: float64)
